In [1]:
import torch
torch.__version__
from torch.nn.utils.parametrizations import orthogonal
import torch.nn.utils.parametrize as P
import torch.nn as nn
from measurements2 import get_surrogate_loss2

In [2]:
class Teacher(nn.Module):
    def __init__(self, n_x, n_t, n_f, n_k, same_x = False):
        super().__init__()
        orth_linear = orthogonal(torch.nn.Linear(n_x, n_f))
        self.U = orth_linear.weight.data
        self.A = torch.randn(n_t, 1, n_f)
        
        self.X = torch.randn(1, n_x, n_k) if same_x else torch.randn(n_t, n_x, n_k)
        self.Y = self.A @ self.U @ self.X

    @property
    def data(self):
        return self.X, self.Y


class Model(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.W_list = nn.ParameterList([nn.Parameter(torch.randn(n_x,n_x)) for i in range(d-1)])
        self.W_list.append(nn.Parameter(torch.randn(n_h,n_x)))
        self.C = nn.Parameter(torch.randn(n_t, 1, n_h))

    def forward(self, X):
        return self.C @ self.weight @ X

    @property
    def weight(self):
        W = None
        for W_ in self.W_list:
            W = W_ if W is None else  W_ @ W
        return W
        
    def eval(self, X, target):
        out = self.forward(X)
        # print(out.shape, target.shape)
        loss = nn.MSELoss()(out, target)
        return loss



In [3]:
def train(epochs):
    optim = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

    for i in range(epochs):
        optim.zero_grad()
        # out = model(X)
        loss = model.eval(X,Y)
        loss.backward()
        optim.step()
        print(i, loss.item())

In [15]:
# width = 'wide'
width = 'narrow'

n_x = 36
n_f = 4 #Feature dim
n_k = 6 #12 # batch-size
n_t = 64# 28 #20 #8  # num task
n_h = n_f if width == 'narrow' else n_x


In [16]:

d = 1
same_x = False
# same_x = True

teacher = Teacher (n_x, n_t, n_f, n_k, same_x = same_x)
X, Y = teacher.data
model = Model(d=d)


# p_list = list(model.parameters())
# print(X.shape, Y.shape)
# print([p.shape for p in p_list])

In [17]:
train(epochs=300)

0 147.41111755371094
1 147.24412536621094
2 146.927490234375
3 146.4771728515625
4 145.90806579589844
5 145.2337188720703
6 144.4666748046875
7 143.61834716796875
8 142.69918823242188
9 141.7186279296875
10 140.685302734375
11 139.60694885253906
12 138.49063110351562
13 137.3426055908203
14 136.16859436035156
15 134.9736785888672
16 133.76239013671875
17 132.5388641357422
18 131.30665588378906
19 130.06898498535156
20 128.82875061035156
21 127.58841705322266
22 126.35025024414062
23 125.11617279052734
24 123.88790893554688
25 122.66696166992188
26 121.45459747314453
27 120.251953125
28 119.05997467041016
29 117.87950897216797
30 116.7112045288086
31 115.55567169189453
32 114.41339874267578
33 113.2847671508789
34 112.17005157470703
35 111.06957244873047
36 109.98343658447266
37 108.91180419921875
38 107.85472869873047
39 106.81226348876953
40 105.78441619873047
41 104.7711181640625
42 103.77233123779297
43 102.78793334960938
44 101.81786346435547
45 100.8619384765625
46 99.920066833496

In [18]:

W=model.weight
U=teacher.U
sl=get_surrogate_loss2(W,U,eps=0)
print(sl)

tensor(0.0764, grad_fn=<SqrtBackward0>)


In [20]:
u, sig, v = W.svd()
W_normalized = u @ v.T

# print(W_normalized@W_normalized.T, U@U.T)

err = W_normalized.T@W_normalized - U.T@U
surrogate_loss = (err.norm()**2/  err.numel()).sqrt()
print(surrogate_loss)

tensor(0.0764, grad_fn=<SqrtBackward0>)


In [12]:
X, Y = teacher.data
# out = model(X)
loss = model.eval(X,Y)
loss.backward()

In [ ]:
X = torch.randn(n_t, n_x, n_k)
a = U @ X
o = C @ a

CU = C@U
# print(a.shape)
# print(o.shape)

In [ ]:
print(CU.shape, C.shape, U.shape)
print(CU.numel(), C.numel(), U.numel())

torch.Size([8, 1, 36]) torch.Size([8, 1, 4]) torch.Size([4, 36])
288 32 144
